# Process Input Scenes Using CGI_HLC_PSFs

In [ ]:
from importlib import reload

import hlc_processing
reload(hlc_processing)

import disk_processing
reload(disk_processing)

import astropy.io.fits as fits
import astropy.units as u
import os
import numpy as np
import poppy.utils as utils
import poppy
import matplotlib.pyplot as plt

%matplotlib inline

from scipy.ndimage import rotate
from matplotlib.colors import LogNorm
import glob
import time
from utils import displ_scale  
from scipy.interpolate import RegularGridInterpolator
import traceback

plt.rcParams.update({'image.origin': 'lower',
                     'image.interpolation':"nearest"})
from pathlib import Path
my_home_dir = os.getcwd() # gets the current working directory
home= os.path.expanduser("~")

In [ ]:
thresh = 1e32
localzodi = 5.2e-11 #Jy/pix (22 mags/sq as)

# Define the resolution and pixel scale
det_oversample = 4
D = 2.3631*u.m
lambda_C = 575*u.nm
resel = ((lambda_C/D)*u.radian).to(u.arcsec) # puts the reselution in arcseconds
print(resel)
out_pixscale = (resel/det_oversample).value


In [ ]:
model_dir = "/Users/edouglas/Box Sync/WFIRST_SIT_FILES/DebesModels/"
dmodels = Path(model_dir).glob('**/*.fits')# glob.glob(Path(my_home_dir+"/DebesModels/*/**.fits"),recursive = True) # collect the debes_models
#for i,debes_model in enumerate(debes_models): print(i,debes_model)
for m in dmodels:
    print(m)


In [ ]:
# Read in the file of PSFs and the file containing the offset values which are in units of Lambda/D at first
psfs_file_name = Path(home+"/projects/WFIRST/data/CGI_HLC_PSFs/" + "20180718_hlc_nfov_PSFs_1Dcube.fits")
print(psfs_file_name)
info_file_name = Path(home+"/projects/WFIRST/data/CGI_HLC_PSFs/" + "20180718_hlc_nfov_PSFs_1Dcube_info.fits")
print(info_file_name)


In [ ]:
# Open the the two fits files
psfs_file = fits.open(psfs_file_name)
info_file = fits.open(info_file_name)

# View the headers of the two files
print("PSFs Header: ")
for i, val in enumerate(psfs_file[0].header,0):
    print(i,val)

print("\nInfo Header: ")
for i, val in enumerate(info_file[0].header,0):
    print(i,val)
    
print("\n")
print(info_file[0].header["SAMPLING"])



In [ ]:
psfs = fits.getdata(psfs_file_name)
psfs_info = fits.getdata(info_file_name)

plt.figure(figsize=[12,12])
i = 30
# move the axis of the PSFs to be compatible with the interpolating function
plt.subplot(121)
plt.imshow(psfs[i,:,:])

# move the axis
psfs = np.moveaxis(psfs,0,-1)

plt.subplot(122)
plt.imshow(psfs[:,:,i])

# Get the offset values which are in the second column of the psfs_info data and convert to arcseconds
offsets_LD = psfs_info[:,1]
offsets_MAS = offsets_LD*((lambda_C/D)*u.radian).to(u.milliarcsecond)

fov_LD = offsets_LD[84]
print(fov_LD)
fov_MAS = offsets_MAS[84]
print(fov_MAS)

In [ ]:
xmas = offsets_MAS

n = 128

x = range(n)
y = range(n)
z = xmas

my_interpolating_function = RegularGridInterpolator((x, y, z), 
                                                    psfs,
                                                    bounds_error = False,
                                                    method = "linear",
                                                    fill_value = 0)

my_interpolating_function_n = RegularGridInterpolator((x, y, z), 
                                                      psfs,
                                                      bounds_error = False,
                                                      method = "nearest", #~2x faster than linear
                                                      fill_value = 0)

In [ ]:
# Now do the interpolation
n = 128

x = range(n)
y = range(n)

grid = np.meshgrid(x,y)
pts = np.vstack([np.vstack([grid[0].flatten(),grid[1].flatten()]),0*np.ones(len(grid[0].flatten()))]).T
interpped = my_interpolating_function(pts).reshape(n,n).T

# Now display everything
plt.figure(figsize=[15,5])

# Plot the original PSF
plt.subplot(131)
plt.title("Original")
plt.imshow(psfs[:,:,0])
plt.colorbar()

# Plot the interpolated PSF
plt.subplot(132)
plt.title("Interpolated")
plt.imshow(interpped)
plt.colorbar()

# Plot the interpolated - original PSF
plt.subplot(133)
plt.title("Residual")
plt.imshow(interpped-psfs[:,:,0])
plt.colorbar()

plt.show()

In [ ]:
# Use the sampling information to get the HLC_plate_scale
HLC_plate_scale_LD = 1/0.2 # get the plate scale in L/D per pixel
HLC_plate_scale_AS = 1/(HLC_plate_scale_LD/resel)
print(HLC_plate_scale_AS)

mask_radius = 0.7/HLC_plate_scale_AS.value
print(mask_radius)

## Make two planets with offsets of 50mas and 100mas.
### The first row is done using linear interpolation and the second row is using nearest interpolation.

In [ ]:
reload(hlc_processing)

plt.figure(figsize = [12,12])

planet1 = hlc_processing.closest_monochrome_PSF(50*u.milliarcsecond,0*u.milliarcsecond,
                                                my_interpolating_function,
                                                xmas,
                                                HLC_plate_scale_AS,
                                                n = 128,
                                                mask_radius = None)

plt.subplot(221)
plt.imshow(planet1)
plt.colorbar()

planet2 = hlc_processing.closest_monochrome_PSF(100*u.milliarcsecond,0*u.milliarcsecond,
                                                my_interpolating_function,
                                                xmas,
                                                HLC_plate_scale_AS,
                                                n = 128,
                                                mask_radius = None)

plt.subplot(222)
plt.imshow(planet2)
plt.colorbar()

planet1_n = hlc_processing.closest_monochrome_PSF(50*u.milliarcsecond,0*u.milliarcsecond,
                                                my_interpolating_function_n,
                                                xmas,
                                                HLC_plate_scale_AS,
                                                n = 128,
                                                mask_radius = None)

plt.subplot(223)
plt.imshow(planet1_n)
plt.colorbar()

planet2_n = hlc_processing.closest_monochrome_PSF(100*u.milliarcsecond,0*u.milliarcsecond,
                                                my_interpolating_function_n,
                                                xmas,
                                                HLC_plate_scale_AS,
                                                n = 128,
                                                mask_radius = None)

plt.subplot(224)
plt.imshow(planet2_n)
plt.colorbar()


## Subtract the nearest from the linearly interpolated planets.

In [ ]:
plt.figure(figsize = [12,6])

planet1_diff = np.abs(planet1-planet1_n)
plt.subplot(121)
plt.imshow(planet1_diff)
plt.colorbar()

planet2_diff = np.abs(planet2-planet2_n)
plt.subplot(122)
plt.imshow(planet2_diff)
plt.colorbar()

max_loc_1 = np.where(planet1_diff == planet1_diff.max())
print("Max difference for planet 1 is {}".format(planet1_diff.max()))
print("Max difference is located at {},{}".format(max_loc_1[0],max_loc_1[1]))
print("The percent difference from the linear interpolation is {}".format(planet1_diff.max()/planet1[max_loc_1]))
#print(planet1[56,64],planet1_n[56,64])

print("\n")

max_loc_2 = np.where(planet2_diff == planet2_diff.max())
print("Max difference for planet 2 is {}".format(planet2_diff.max()))
print("Max difference is located at {},{}".format(max_loc_2[0],max_loc_2[1]))
print("The percent difference from the linear interpolation is {}".format(planet2_diff.max()/planet2[max_loc_2]))
#print(planet2[51,64],planet2_n[51,64])


## Save the 100mas planet to a fits file

In [ ]:
plt.figure(figsize=[6,6])

planet_100mas = hlc_processing.closest_monochrome_PSF(100*u.milliarcsecond,0*u.milliarcsecond,
                                                      my_interpolating_function,
                                                      xmas,
                                                      HLC_plate_scale_AS,
                                                      n=128,
                                                      mask_radius = None)

plt.imshow(planet_100mas)
plt.colorbar()

planet_fits_name = my_home_dir + "\planets\\" + "planet_100mas_CGI_HLC.fits"
fits.writeto(planet_fits_name,planet_100mas,overwrite=True)

## Side by side comparison of planets using CGI vs OS5

In [ ]:
planet_fits_name_CGI = Path(my_home_dir + "/planets/" + "planet_100mas_CGI_HLC.fits")
planet_CGI = fits.getdata(planet_fits_name_CGI)

planet_fits_name_OS5 = Path(my_home_dir + "/planets/" + "planet_100mas_OS5_3.2.fits")
planet_OS5 = fits.getdata(planet_fits_name_OS5)

plt.figure(figsize = [12,5])
plt.subplot(121)
plt.imshow(planet_CGI)
plt.title("CGI 1D Cube PSFs")
plt.colorbar()

plt.subplot(122)
plt.imshow(planet_OS5)
plt.title("OS5 PSFs")
plt.colorbar()


## Define the paths for the zodi files

In [ ]:
zodi_path = Path(my_home_dir + "/zodipics/")
print(zodi_path)

output_path_10pc03mas01alpha =Path(my_home_dir + "/zodipics_HLC_outputs/" + "zodipic_10pc03mas01alpha/")
print(output_path_10pc03mas01alpha)

output_path_10pc03mas = Path(my_home_dir + "/zodipics_HLC_outputs/" + "zodipic_10pc03mas/")
print(output_path_10pc03mas)

output_path_03pc03mas01alpha = Path(my_home_dir + "/zodipics_HLC_outputs/" + "zodipic_03pc03mas01alpha/")
print(output_path_03pc03mas01alpha)

output_path_03pc03mas = Path(my_home_dir + "/zodipics_HLC_outputs/" + "zodipic_03pc03mas/")
print(output_path_03pc03mas)

# Also define mask and core_mask radii
core_mask_radius = .15*u.arcsec # 150mas
mask_radius = mask_radius

## zodipic_\*pc\*mas\*.fits

### Output files:
zodipic_\*pc\*mas\*\_linear_HLC.fits

zodipic_\*pc\*mas\*\_linear_input.fits

zodipic_\*pc\*mas\*\_linear_coremaksed_HLC.fits

zodipic_\*pc\*mas\*\_linear_coremaksed_input.fits


In [ ]:
reload(hlc_processing)

zodi_fname = Path("zodipic_10pc03mas01alpha.fits")
zodi_file = zodi_path / Path(zodi_fname)

im_name = output_path_10pc03mas01alpha / (zodi_fname[:-5]  "_linear_HLC.fits")
print(im_name)

zodi_name = output_path_10pc03mas01alpha / (zodi_fname[:-5] + "_linear_input.fits")
print(zodi_name)

start_time = time.time()
    
im,zodi = hlc_processing.fit_to_hlc(zodi_file,
                                    my_interpolating_function,
                                    xmas,
                                    HLC_plate_scale_AS,
                                    n = 128,
                                    thresh = thresh,
                                    display=True)

zodi_header = fits.getheader(zodi_file)
zodi_header["thresh"] = thresh
zodi_header["coremask"] = core_mask_radius.value

fits.writeto(im_name, im, overwrite=True, header=zodi_header) 
fits.writeto(zodi_name, zodi.filled(fill_value = 0), overwrite = True)

print(time.time()-start_time)

In [ ]:
reload(hlc_processing)

zodi_fname = "zodipic_10pc03mas01alpha.fits"
zodi_file = zodi_path / zodi_fname

im_name = Path(output_path_10pc03mas01alpha / zodi_fname[:-5]) / "_linear_coremasked_HLC.fits"
print(im_name)

zodi_name = Path(output_path_10pc03mas01alpha / zodi_fname[:-5]) / "_linear_coremasked_input.fits"
print(zodi_name)

start_time = time.time()
    
im,zodi = hlc_processing.fit_to_hlc(zodi_file,
                                    my_interpolating_function,
                                    xmas,
                                    HLC_plate_scale_AS,
                                    n = 128,
                                    thresh = thresh,
                                    display=True,
                                    core_mask_radius=core_mask_radius)

zodi_header = fits.getheader(zodi_file)
zodi_header["thresh"] = thresh
zodi_header["coremask"] = core_mask_radius.value

fits.writeto(im_name, im, overwrite=True, header=zodi_header) 
fits.writeto(zodi_name, zodi.filled(fill_value = 0), overwrite = True)

print(time.time()-start_time)

## Compute difference between the linear and nearest interpolations for zodipic_10pc03mas01alpha.fits

In [ ]:
zodi_fname = "zodipic_10pc03mas01alpha.fits"

im_name = output_path_10pc03mas01alpha + zodi_fname[:-5] + "_HLC.fits"
print(im_name)

im_l_name = output_path_10pc03mas01alpha + zodi_fname[:-5] + "_linear_HLC.fits"
print(im_l_name)

im = fits.getdata(im_name)
im_l = fits.getdata(im_l_name)

diff = np.abs(im-im_l)

plt.figure(figsize = [4,4])
plt.imshow(diff)
plt.colorbar()

max_loc = np.where(diff == diff.max())
print("\nMax difference for planet 1 is {}".format(diff.max()))
print("Max difference is located at {},{}".format(max_loc[0],max_loc[1]))
print("The percent difference from the linear interpolation is {}%".format(100*diff.max()/im_l[max_loc]))


In [ ]:
im_name = output_path_10pc03mas01alpha + zodi_fname[:-5] + "_coremasked_HLC.fits"
print(im_name)

im_l_name = output_path_10pc03mas01alpha + zodi_fname[:-5] + "_linear_coremasked_HLC.fits"
print(im_l_name)

im = fits.getdata(im_name)
im_l = fits.getdata(im_l_name)

diff = np.abs(im-im_l)

plt.figure(figsize = [4,4])
plt.imshow(diff)
plt.colorbar()

max_loc = np.where(diff == diff.max())
print("\nMax difference for planet 1 is {}".format(diff.max()))
print("Max difference is located at {},{}".format(max_loc[0],max_loc[1]))
print("The percent difference from the linear interpolation is {}%".format(100*diff.max()/im_l[max_loc]))


# Now to run the Debes Models

## First run Model_Annulus\annulus_inc60_r0.4.fits as an example

In [ ]:
# Run it using linear first
reload(disk_processing)

thresh = 5

fitsfile = my_home_dir + "\DebesModels\\" + "Model_Annulus\\" + "annulus_inc60_r0.4.fits"

start_time = time.time()

im,zodi,outfits = disk_processing.disk_through_hlc(fitsfile,
                                                   my_interpolating_function,
                                                   xmas,
                                                   HLC_plate_scale_AS,
                                                   n = 128,
                                                   thresh=5,
                                                   display=True)

fits.writeto(outfits, im, overwrite = True)

end_time = time.time()
print(end_time-start_time)

In [ ]:
# Now run it using nearest
reload(disk_processing)

thresh = 5

fitsfile = my_home_dir + "\DebesModels\\" + "Model_Annulus\\" + "annulus_inc60_r0.4.fits"

start_time = time.time()

im,zodi,outfits = disk_processing.disk_through_hlc(fitsfile,
                                                   my_interpolating_function_n,
                                                   xmas,
                                                   HLC_plate_scale_AS,
                                                   n=128,
                                                   thresh=5,
                                                   display=True)

fitsfile_parts = fitsfile.split("\\")
outfits = my_home_dir + "\KianDebesModels_HLC\\" + fitsfile_parts[-2] + "\\" + fitsfile_parts[-1][:-5] + "_HLC_" + str(thresh) + '_nearest.fits'

fits.writeto(outfits, im, overwrite = True)

end_time = time.time()
print(end_time-start_time)

In [ ]:
# Compare the linear and nearest interpolation results
fitsfile = my_home_dir + "\DebesModels\\" + "Model_Annulus\\" + "annulus_inc60_r0.4.fits"

fitsfile_parts = fitsfile.split("\\")

thresh = 5

outfits_n = my_home_dir + "\KianDebesModels_HLC\\" + fitsfile_parts[-2] + "\\" + fitsfile_parts[-1][:-5] + "_HLC_" + str(thresh) + '_nearest.fits'
outfits_l = my_home_dir + "\KianDebesModels_HLC\\" + fitsfile_parts[-2] + "\\" + fitsfile_parts[-1][:-5] + "_HLC_" + str(thresh) + '.fits'

im_n = fits.getdata(outfits_n)
im_l = fits.getdata(outfits_l)

diff = np.abs(im_n - im_l)

print(diff.max())

max_loc = np.where(diff == diff.max())
print("Max location in difference array is {},{}".format(max_loc[0],max_loc[1]))
print("Percentage of linearly interpped at the max differnce location is {}%".format(100*diff.max()/im_l[max_loc]))

plt.figure(figsize = [15,6])

plt.subplot(131)
plt.imshow(im_n)
plt.title("Nearest")
plt.colorbar()

plt.subplot(132)
plt.imshow(im_l)
plt.title("Linear")
plt.colorbar()

plt.subplot(133)
plt.imshow(diff)
plt.title("Absolute Differnce")
plt.colorbar()

In [ ]:
# Compare with that from OS5 PSFs

fitsfile = my_home_dir + "\DebesModels\\" + "Model_Annulus\\" + "annulus_inc60_r0.4.fits"
fitsfile_parts = fitsfile.split("\\")

outfits_HLC = my_home_dir + "\KianDebesModels_HLC\\" + fitsfile_parts[-2] + "\\" + fitsfile_parts[-1][:-5] + "_HLC_5.fits"
outfits_OS5 = my_home_dir + "\KianDebesModels_OS5\\" + fitsfile_parts[-2] + "\\" + fitsfile_parts[-1][:-5] + '_HLC_5.fits'

im_HLC = fits.getdata(outfits_HLC)
im_OS5 = fits.getdata(outfits_OS5)

plt.figure(figsize = [15,6])

plt.subplot(121)
plt.imshow(im_HLC)
plt.title("HLC (linear)")
plt.colorbar()

plt.subplot(122)
plt.imshow(im_OS5.T)
plt.title("OS5")
plt.colorbar()


# Now just run all the Debes Models

In [ ]:
# Separate the files into the annulus, constanttau, gap, and ring files
dmodels_A = list()
dmodels_C = list()
dmodels_G = list()
dmodels_R = list()

for i,model in enumerate(dmodels):
    if "Annulus" in model: 
        dmodels_A.append(model)
    if "ConstantTau" in model: 
        dmodels_C.append(model)
    if "GAP" in model: 
        dmodels_G.append(model)
    if "Ring" in model: 
        dmodels_R.append(model)

dmodels_G

In [ ]:
dmodels_A_inc0 = list()
dmodels_A_inc30 = list()
dmodels_A_inc45 = list()
dmodels_A_inc60 = list()
dmodels_A_inc89 = list()

for i,model in enumerate(dmodels_A):
    if "inc0" in model:
        dmodels_A_inc0.append(model)
    if "inc30" in model:
        dmodels_A_inc30.append(model)
    if "inc45" in model:
        dmodels_A_inc45.append(model)
    if "inc60" in model:
        dmodels_A_inc60.append(model)
    if "inc89" in model:
        dmodels_A_inc89.append(model)

dmodels_A_inc30

In [ ]:
dmodels_C_inc0 = list()
dmodels_C_inc30 = list()
dmodels_C_inc45 = list()
dmodels_C_inc60 = list()
dmodels_C_inc89 = list()

for i,model in enumerate(dmodels_C):
    if "inc0" in model:
        dmodels_C_inc0.append(model)
    if "inc30" in model:
        dmodels_C_inc30.append(model)
    if "inc45" in model:
        dmodels_C_inc45.append(model)
    if "inc60" in model:
        dmodels_C_inc60.append(model)
    if "inc89" in model:
        dmodels_C_inc89.append(model)

dmodels_C_inc30

In [ ]:
dmodels_R_inc0 = list()
dmodels_R_inc30 = list()
dmodels_R_inc45 = list()
dmodels_R_inc60 = list()
dmodels_R_inc89 = list()

for i,model in enumerate(dmodels_R):
    if "inc0" in model:
        dmodels_R_inc0.append(model)
    if "inc30" in model:
        dmodels_R_inc30.append(model)
    if "inc45" in model:
        dmodels_R_inc45.append(model)
    if "inc60" in model:
        dmodels_R_inc60.append(model)
    if "inc89" in model:
        dmodels_R_inc89.append(model)

dmodels_R_inc30

In [ ]:
reload(disk_processing)

start_time = time.time()

for i,fitsfile in enumerate(dmodels_G):
    
    start = time.time()
    print("\nRunning file number {}.".format(i+1))
    
    im,zodi,outfits = disk_processing.disk_through_hlc(fitsfile,
                                                       my_interpolating_function,
                                                       xmas,
                                                       HLC_plate_scale_AS,
                                                       n = 128,
                                                       thresh=5,
                                                       display=True)
    
    fits.writeto(outfits, im, overwrite = True)
    end = time.time()
    print(end-start)
    
end_time = time.time()

print(str(end_time - start_time) + "\n")